In [1]:
import calendar
import pandas as pd
import panel as pn

from datetime import date, timedelta
from sqlalchemy import create_engine

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 12)

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trade name qty target active reason market'.split()

In [3]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite)

,id,trade,name,qty,price,active,reason,market
0,700,B,KKP,3000,68,0,3L,SET100


### After call Ord-Log.bat which calculates spreads between target price and SET price.

In [4]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
#col_names   = ['trade','name','spreads','reason','market','qty','target_price','set_price','chg_amt','chg_pct','active']
df_log = pd.read_csv(input_file, sep=',',  index_col=None)
df_log

,trade,name,spd,reason,market,qty,target,current,change,percent,active
0,B,ASP,-5,6L,SET,30000,3.72,3.82,0.00,0.00%,0
1,B,BGC,-5,DOS,SET,10000,10.00,10.50,0.00,0.00%,0
2,B,CPNCG,-1,DOS,SET,10000,12.60,12.70,0.00,0.00%,0
3,B,KKP,-11,3L,SET100,3000,68.00,70.75,-0.25,-0.35%,0
4,B,MEGA,-5,1L,SET100,4500,43.00,44.25,-1.00,-2.21%,0
5,B,NOBLE,-9,RD15%,SET,21000,5.85,6.30,0.05,+0.80%,0
6,B,RATCH,-13,RD05%,SET50,3000,42.75,46.00,0.00,0.00%,0
7,B,ROJNA,-4,6L,SET,18000,6.30,6.50,-0.10,-1.52%,0
8,B,SCC,-13,RD05pct,SET50,300,374.00,387.00,-4.00,-1.02%,0
9,B,SINGER,-3,3L,SET100,4500,42.50,43.25,-0.50,-1.14%,0


In [5]:
def calc(vals):
    spreads = vals
    if abs(spreads) < 10:
        return 1
    else:
        return 0

In [6]:
df_log['active'] = df_log['spd'].apply(calc)

In [7]:
df_log.groupby('active')['trade'].count()

active
0     7
1    23
Name: trade, dtype: int64

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

31

In [9]:
df_log.dtypes

trade       object
name        object
spd          int64
reason      object
market      object
qty          int64
target     float64
current    float64
change     float64
percent     object
active       int64
dtype: object

In [10]:
final = df_log[colt]
final

,trade,name,qty,target,active,reason,market
0,B,ASP,30000,3.72,1,6L,SET
1,B,BGC,10000,10.00,1,DOS,SET
2,B,CPNCG,10000,12.60,1,DOS,SET
3,B,KKP,3000,68.00,0,3L,SET100
4,B,MEGA,4500,43.00,1,1L,SET100
5,B,NOBLE,21000,5.85,1,RD15%,SET
6,B,RATCH,3000,42.75,0,RD05%,SET50
7,B,ROJNA,18000,6.30,1,6L,SET
8,B,SCC,300,374.00,0,RD05pct,SET50
9,B,SINGER,4500,42.50,1,3L,SET100


In [11]:
rcds = final.values.tolist()
rcds

[['B', 'ASP', 30000, 3.72, 1, '6L', 'SET'],
 ['B', 'BGC', 10000, 10.0, 1, 'DOS', 'SET'],
 ['B', 'CPNCG', 10000, 12.6, 1, 'DOS', 'SET'],
 ['B', 'KKP', 3000, 68.0, 0, '3L', 'SET100'],
 ['B', 'MEGA', 4500, 43.0, 1, '1L', 'SET100'],
 ['B', 'NOBLE', 21000, 5.85, 1, 'RD15%', 'SET'],
 ['B', 'RATCH', 3000, 42.75, 0, 'RD05%', 'SET50'],
 ['B', 'ROJNA', 18000, 6.3, 1, '6L', 'SET'],
 ['B', 'SCC', 300, 374.0, 0, 'RD05pct', 'SET50'],
 ['B', 'SINGER', 4500, 42.5, 1, '3L', 'SET100'],
 ['B', 'SPALI', 9000, 21.6, 0, '3L', 'SET100'],
 ['B', 'STA', 2500, 30.25, 1, 'RD10pct', 'SET100'],
 ['B', 'TISCO', 2000, 98.0, 1, 'CP1S', 'SET50'],
 ['B', 'TQM', 4500, 45.0, 1, '3L', 'SET100'],
 ['B', 'TVO', 3600, 30.0, 1, 'DOS', 'SET100'],
 ['S', 'CKP', 10000, 5.25, 1, '5pct', 'SET100'],
 ['S', 'DIF', 10000, 14.3, 1, 'CUT', 'SET'],
 ['S', 'EPG', 6000, 10.5, 1, 'CUT', 'SET100'],
 ['S', 'GLOBAL', 7500, 20.4, 1, '5pct', 'SET50'],
 ['S', 'IP', 2500, 21.2, 1, '5pct', 'mai'],
 ['S', 'IVL', 1200, 52.5, 1, '25pct', 'SET50'],
 [

In [12]:
for rcd in rcds:
    print(rcd)

['B', 'ASP', 30000, 3.72, 1, '6L', 'SET']
['B', 'BGC', 10000, 10.0, 1, 'DOS', 'SET']
['B', 'CPNCG', 10000, 12.6, 1, 'DOS', 'SET']
['B', 'KKP', 3000, 68.0, 0, '3L', 'SET100']
['B', 'MEGA', 4500, 43.0, 1, '1L', 'SET100']
['B', 'NOBLE', 21000, 5.85, 1, 'RD15%', 'SET']
['B', 'RATCH', 3000, 42.75, 0, 'RD05%', 'SET50']
['B', 'ROJNA', 18000, 6.3, 1, '6L', 'SET']
['B', 'SCC', 300, 374.0, 0, 'RD05pct', 'SET50']
['B', 'SINGER', 4500, 42.5, 1, '3L', 'SET100']
['B', 'SPALI', 9000, 21.6, 0, '3L', 'SET100']
['B', 'STA', 2500, 30.25, 1, 'RD10pct', 'SET100']
['B', 'TISCO', 2000, 98.0, 1, 'CP1S', 'SET50']
['B', 'TQM', 4500, 45.0, 1, '3L', 'SET100']
['B', 'TVO', 3600, 30.0, 1, 'DOS', 'SET100']
['S', 'CKP', 10000, 5.25, 1, '5pct', 'SET100']
['S', 'DIF', 10000, 14.3, 1, 'CUT', 'SET']
['S', 'EPG', 6000, 10.5, 1, 'CUT', 'SET100']
['S', 'GLOBAL', 7500, 20.4, 1, '5pct', 'SET50']
['S', 'IP', 2500, 21.2, 1, '5pct', 'mai']
['S', 'IVL', 1200, 52.5, 1, '25pct', 'SET50']
['S', 'NER', 9000, 7.8, 0, '5pct', 'SET']
['

In [13]:
sql = """
INSERT INTO orders (trade, name, qty, price, active, reason, market)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active, reason, market)
VALUES (?, ?, ?, ?, ?, ?, ?)



In [14]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [15]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(df_orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=    id trade  ..., width=800)

In [16]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)